In [75]:
import pandas as pd
import numpy as np


In [76]:
df= pd.read_excel('Dataset.xlsx') 


In [77]:
df.head()

,SL,Emotion,Text,Year,Racial Biases,Hatred Percentage,Depression Percentage
0,1,disgust,নির্বাচনের আগের রাতে সুষ্টু শান্তিপূর্ণ ভাবে ভ...,2016,NaN,NaN,NaN
1,2,disgust,জনগণ মনের আনন্দে নিজেদের ভোট কাস্ট করে দেশদ্রো...,2016,NaN,NaN,NaN
2,3,angry,ভেজাল নির্বাচন ।,2016,NaN,NaN,NaN
3,4,happy,বিএনপি ও রাজাকার সমর্থকরা হেরে গেলে দেশের জন্য...,2016,NaN,NaN,NaN
4,5,angry,"সূক্ষ্ম কারচুপি বলে এতদিন একটা কথার কথা ছিল, এ...",2016,NaN,NaN,NaN


In [78]:
df.tail()

,SL,Emotion,Text,Year,Racial Biases,Hatred Percentage,Depression Percentage
35995,35996,surprise,ভারতের মতো একটা বিশাল দেশে এত অঙ্গরাজ্য যে সব ...,2021,NaN,NaN,NaN
35996,35997,surprise,"২৪, ৫০, এমনকি কারো কারো মতে ৬০ জন।",2021,NaN,NaN,NaN
35997,35998,surprise,এদের মধ্য থেকে ২৭ জনকে রাজা বিয়ে দিয়েছিল চন্দ্...,2021,NaN,NaN,NaN
35998,35999,surprise,"ওয়াকফের শাব্দিক অর্থ কোনো কিছু আটকে রাখা, উৎসর...",2021,NaN,NaN,NaN
35999,36000,surprise,তৃণমূলের সাবেক মন্ত্রী শুভেন্দু সম্প্রতি মমতা...,2021,NaN,NaN,NaN


In [79]:
df["Emotion"].value_counts()

disgust     6000
angry       6000
happy       6000
surprise    6000
fear        6000
sad         6000
Name: Emotion, dtype: int64

In [80]:
class_names = ['disgust','angry','happy','surprise','fear','sad']

In [7]:
encoding = {
    'disgust': 0, 
    'angry' :1,
    'happy':2,
    'surprise':3,
    'fear':4,
    'sad':5
}

In [81]:
text = df['Text'].values.tolist()
emotion= df['Emotion'].values.tolist()

In [82]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(text , emotion, test_size=.2)

In [10]:
y_train = [encoding[x] for x in y_train]
y_test = [encoding[x] for x in y_test]

In [84]:
df["Emotion"].value_counts()

disgust     6000
angry       6000
happy       6000
surprise    6000
fear        6000
sad         6000
Name: Emotion, dtype: int64

In [85]:
import ktrain
from ktrain import text
MODEL_NAME = 'sagorsarker/bangla-bert-base'
t = text.Transformer(MODEL_NAME, maxlen=500)


In [86]:
trn = t.preprocess_train(X_train, y_train)
val = t.preprocess_test(X_test, y_test)

preprocessing train...
language: bn
train sequence lengths:
	mean : 16
	95percentile : 44
	99percentile : 66


Is Multi-Label? False
preprocessing test...
language: bn
test sequence lengths:
	mean : 16
	95percentile : 45
	99percentile : 70


In [87]:
model = t.get_classifier()

In [88]:
learner = ktrain.get_learner(model, train_data=trn, val_data=val, batch_size=6)

In [ ]:
learner.lr_find(show_plot=True, max_epochs=1)

In [89]:
learner.fit_onecycle(2e-5, 3)



begin training using onecycle policy with max lr of 2e-05...
Epoch 1/3
4800/4800 [==============================] - 92237s 19s/step - loss: 1.3248 - accuracy: 0.4865 - val_loss: 0.8402 - val_accuracy: 0.7101
Epoch 2/3
4800/4800 [==============================] - 93490s 19s/step - loss: 0.6400 - accuracy: 0.7875 - val_loss: 0.5421 - val_accuracy: 0.8247
Epoch 3/3
4800/4800 [==============================] - 92498s 19s/step - loss: 0.2784 - accuracy: 0.9096 - val_loss: 0.4559 - val_accuracy: 0.8617


In [90]:
learner.validate(val_data=(val), class_names=class_names)

              precision    recall  f1-score   support

     disgust       0.83      0.86      0.84      1194
       angry       0.89      0.88      0.88      1182
       happy       0.92      0.91      0.92      1263
    surprise       0.88      0.88      0.88      1200
        fear       0.77      0.75      0.76      1234
         sad       0.88      0.90      0.89      1127

    accuracy                           0.86      7200
   macro avg       0.86      0.86      0.86      7200
weighted avg       0.86      0.86      0.86      7200



array([[1026,   30,   19,   21,   79,   19],
       [  52, 1039,    8,   14,   49,   20],
       [  28,    8, 1152,   17,   42,   16],
       [  27,   13,   16, 1050,   59,   35],
       [  95,   61,   42,   64,  923,   49],
       [  15,   17,   13,   23,   45, 1014]], dtype=int64)

In [91]:
learner.evaluate()


              precision    recall  f1-score   support

           0       0.83      0.86      0.84      1194
           1       0.89      0.88      0.88      1182
           2       0.92      0.91      0.92      1263
           3       0.88      0.88      0.88      1200
           4       0.77      0.75      0.76      1234
           5       0.88      0.90      0.89      1127

    accuracy                           0.86      7200
   macro avg       0.86      0.86      0.86      7200
weighted avg       0.86      0.86      0.86      7200



array([[1026,   30,   19,   21,   79,   19],
       [  52, 1039,    8,   14,   49,   20],
       [  28,    8, 1152,   17,   42,   16],
       [  27,   13,   16, 1050,   59,   35],
       [  95,   61,   42,   64,  923,   49],
       [  15,   17,   13,   23,   45, 1014]], dtype=int64)

In [93]:
model.save("Bangla-Bert_model")